In [ ]:
!pip install -q tqdm tensorflow tensorflow_addons transformers pandas numpy shap emoji==0.6.0

In [ ]:
import sys
get_ipython().system('{sys.executable} -m pip install seaborn')
import csv
import pandas as pd
import matplotlib.pyplot as plt
from numpy import argmax
import seaborn as sns
import tensorflow as tf
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import ZeroShotClassificationPipeline
import shap
from typing import Union, List
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import emoji
import datetime

In [ ]:

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary


task='emotion'
# MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
MODEL = f"Emanuel/bertweet-emotion-base"


# PT
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/danielhamelberg/nlp/main/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [ ]:
df = pd.read_csv('/content/text_without_emojis.csv', usecols=['text'])
df['text'] = df['text'].fillna('')

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = [] # Why do we encounter an error about 'Series' object has no attribute 'split'? Answer: because we are passing a Series object, not a string. We need to iterate over the Series object:
    for i in text:
        i = i.replace('@USER', '<user>')
        i = i.replace('URL', '<url>')
        new_text.append(i)
    # for t in text.split(" "):
    #     t = '@user' if t.startswith('@') and len(t) > 1 else t
    #     t = 'http' if t.startswith('http') else t
    #     new_text.append(t)
    return " ".join(new_text)
# Create a function that feeds df['text'] to the preprocess function.
# df['text'] = df['text'].apply(preprocess) # Apply the preprocess function to the text column of the dataframe


In [ ]:
# TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL, from_pt=True)
# model.save_pretrained(MODEL)
# TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
def run(df):
    row_count = df.shape[0]
    for i, row in df.iterrows():
        # TF
        encoded_input = tokenizer(row['text'], return_tensors='tf', truncation=True) 
        output = model(encoded_input) # Here we want to use the model to predict the output
        scores = output[0][0].numpy() # Here we want to get the scores from the output
        scores = softmax(scores) # Here we want to apply the softmax function to the scores
        # Here we want to save the highest score and the corresponding label to the dataframe (df) while respecting the order of the labels and rows
        df.loc[i, 'score'] = scores[0] # Here we want to save the highest score to the dataframe (df)
        df.loc[i, 'label'] = labels[0] # Here we want to save the corresponding label to the dataframe (df)
        if i % 100 == 0:
            # Write the dataframe to a csv file, appending the number rows at a time.
            # mode = 'a' means append, header = False means don't write the header again.
            # df.to_csv('result.csv', index=True, mode='a', header=False)
            df.to_csv('result.csv', index=True, header=False)
            print(f"{i}/{row_count} rows written to csv")
    return df
run(df)

0/13697 rows written to csv
10/13697 rows written to csv
20/13697 rows written to csv
30/13697 rows written to csv
40/13697 rows written to csv
50/13697 rows written to csv
60/13697 rows written to csv
70/13697 rows written to csv
80/13697 rows written to csv
90/13697 rows written to csv
100/13697 rows written to csv
110/13697 rows written to csv
120/13697 rows written to csv
130/13697 rows written to csv
140/13697 rows written to csv
150/13697 rows written to csv
160/13697 rows written to csv
170/13697 rows written to csv
180/13697 rows written to csv
190/13697 rows written to csv
200/13697 rows written to csv
210/13697 rows written to csv
220/13697 rows written to csv
230/13697 rows written to csv
240/13697 rows written to csv
250/13697 rows written to csv
260/13697 rows written to csv
270/13697 rows written to csv
280/13697 rows written to csv
290/13697 rows written to csv
300/13697 rows written to csv
310/13697 rows written to csv
320/13697 rows written to csv
330/13697 rows writte

In [ ]:
# def run():
#   for i, row in df.iterrows():
#     encoded_input = tokenizer(row['text'], return_tensors='tf')
#     output = model(encoded_input)
#     scores = output[0][0].numpy()
#     scores = softmax(scores)

#     ranking = np.argsort(scores)
#     ranking = ranking[::-1]
#     for i in range(scores.shape[0]):
#       l = labels[ranking[i]]
#       s = scores[ranking[i]]
#       df['result'].append(np.round(float(s), 4))

#       return row['text'], f"{i+1}) {l} {np.round(float(s), 4)}"


# run()

<function __main__.run()>